In [20]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split 
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [21]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [22]:
df = pd.read_csv("D://M_AI//Natural Language Processing//Datasets//Sentiment.csv")
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [23]:
df.shape

(13871, 21)

In [24]:
df.sentiment.value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [25]:
#Taking only text and sentiment columns into consideration
df = df[['text','sentiment']]
df.shape

(13871, 2)

In [26]:
df.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [27]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [28]:
#Taking only positive and negative reviews
df = df[df.sentiment != "Neutral"]

In [29]:
df.text

1        RT @ScottWalker: Didn't catch the full #GOPdeb...
3        RT @RobGeorge: That Carly Fiorina is trending ...
4        RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
5        RT @GregAbbott_TX: @TedCruz: "On my first day ...
6        RT @warriorwoman91: I liked her and was happy ...
                               ...                        
13866    RT @cappy_yarbrough: Love to see men who will ...
13867    RT @georgehenryw: Who thought Huckabee exceede...
13868    RT @Lrihendry: #TedCruz As President, I will a...
13869    RT @JRehling: #GOPDebate Donald Trump says tha...
13870    RT @Lrihendry: #TedCruz headed into the Presid...
Name: text, Length: 10729, dtype: object

In [31]:
#Removing "RT" letters from the trext column
df['text'] = df['text'].str.replace("RT"," ")

In [32]:
df.text

1          @ScottWalker: Didn't catch the full #GOPdeba...
3          @RobGeorge: That Carly Fiorina is trending -...
4          @DanScavino: #GOPDebate w/ @realDonaldTrump ...
5          @GregAbbott_TX: @TedCruz: "On my first day I...
6          @warriorwoman91: I liked her and was happy w...
                               ...                        
13866      @cappy_yarbrough: Love to see men who will n...
13867      @georgehenryw: Who thought Huckabee exceeded...
13868      @Lrihendry: #TedCruz As President, I will al...
13869      @JRehling: #GOPDebate Donald Trump says that...
13870      @Lrihendry: #TedCruz headed into the Preside...
Name: text, Length: 10729, dtype: object

In [34]:
#Removing "@" and "#" letters from the text column
df['text'] = df['text'].str.replace("@"," ")
df['text'] = df['text'].str.replace("#"," ")
df.text

1           ScottWalker: Didn't catch the full  GOPdeba...
3           RobGeorge: That Carly Fiorina is trending -...
4           DanScavino:  GOPDebate w/  realDonaldTrump ...
5           GregAbbott_TX:  TedCruz: "On my first day I...
6           warriorwoman91: I liked her and was happy w...
                               ...                        
13866       cappy_yarbrough: Love to see men who will n...
13867       georgehenryw: Who thought Huckabee exceeded...
13868       Lrihendry:  TedCruz As President, I will al...
13869       JRehling:  GOPDebate Donald Trump says that...
13870       Lrihendry:  TedCruz headed into the Preside...
Name: text, Length: 10729, dtype: object

In [35]:
ps = nltk.PorterStemmer()

In [38]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+',text)
    text = [ps.stem(word) for word in tokens if word not in stopwords.words('english')]
    return text

In [39]:
tfidf = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf.fit_transform(df['text'])

In [40]:
feature_names = tfidf.get_feature_names()
pd.DataFrame(X_tfidf.toarray(), columns=feature_names)

,,0,07,0787,0iiiiiii0girl,1,10,100,1000,10000,...,zombiereaganris,zone,zone6combat,zonkerpa,zoo,zuckerberg,zwigzag,zygot,zzcrane,⁰capabl
0,0.116117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.124570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.130304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.054980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.146709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10724,0.073827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10725,0.110727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10726,0.092933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10727,0.132597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
X_features = pd.DataFrame(X_tfidf.toarray())

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_features, df['sentiment'], train_size=0.8, random_state= 123)

In [44]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8583, 12861)
(2146, 12861)
(8583,)
(2146,)


In [45]:
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
rf_model = rf.fit(X_train,y_train)

In [46]:
y_pred = rf_model.predict(X_test)
print(confusion_matrix(y_test,y_pred))

[[1633   40]
 [ 291  182]]


In [47]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    Negative       0.85      0.98      0.91      1673
    Positive       0.82      0.38      0.52       473

    accuracy                           0.85      2146
   macro avg       0.83      0.68      0.72      2146
weighted avg       0.84      0.85      0.82      2146



In [55]:
y_pred_proba = rf_model.predict_proba(X_test)
y_pred_proba

array([[0.87666667, 0.12333333],
       [0.48791667, 0.51208333],
       [0.77379679, 0.22620321],
       ...,
       [0.73      , 0.27      ],
       [0.965     , 0.035     ],
       [0.855     , 0.145     ]])

In [50]:
y_pred_train = rf_model.predict(X_train)
print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

    Negative       0.98      0.99      0.99      6820
    Positive       0.96      0.94      0.95      1763

    accuracy                           0.98      8583
   macro avg       0.97      0.96      0.97      8583
weighted avg       0.98      0.98      0.98      8583



In [53]:
from sklearn.naive_bayes import GaussianNB
gu = GaussianNB()
gu_model = br.fit(X_train,y_train)

In [54]:
y_pred1 = gu_model.predict(X_test)
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

    Negative       0.84      0.95      0.89      1673
    Positive       0.68      0.38      0.49       473

    accuracy                           0.82      2146
   macro avg       0.76      0.66      0.69      2146
weighted avg       0.81      0.82      0.80      2146

